# FirecREST Tutorial

To run a jupyter cell click on it and then press CTRL+Enter. Let's try the first one to make sure we have the necessary packages.

In [ ]:
import json
import requests
import time

In [ ]:
# If that doesn't work you may need to install requests package:
!pip install --user requests

Let's setup the credentials, as well as the FirecREST and Object Storage IPs, as we will use it many times during the tutorial.

In [ ]:
TOKEN='<token>'
FIRECREST_IP = 'http://148.187.98.88:8000'

## Test the credentials with a simple call

In [ ]:
response = requests.get(
    url=f'{FIRECREST_IP}/status/systems',
    headers={'Authorization': f'Bearer {TOKEN}'}
)

print(response.status_code)
print(response.headers)
print(response.json())

We can take the three parts of the response:

1. the status code by `response.status_code`
2. the headers by `response.headers`
3. the json part by calling `response.json()`

But the output is not very readable, so now we are going to define a function that will print the response in a more readable way.

Run the next cell to get the output in better format.

In [ ]:
# This function is useful only to print the response in a nicer way
def handle_response(response):
    print("\nResponse status code:")
    print(response.status_code)
    print("\nResponse headers:")
    print(json.dumps(dict(response.headers), indent=4))
    print("\nResponse json:")
    try:
        print(json.dumps(response.json(), indent=4))
    except json.JSONDecodeError:
        print("-")

handle_response(response)

## List the contents of a directory

Let's move on to a basic `GET` call on the api.

For start let's set only the required fields:

- `targetPath`, which will be the location in the machine's filesystem

- `X-Machine-Name`, the filesystem we are interested in

- the `Authorization` token


### `GET /utilities/ls`

```ini
Query Parameters:
    targetPath (string) : Absolute filesystem path (Required)
    showhidden (boolean) : do not ignore entries starting with ‘.’
    pageSize (integer) : Number of entries returned
    pageNumber (integer) : Page number  
Status Codes:
    200 OK : List of contents of path
    400 Bad Request : Error listing contents of path 
Request Headers:
    Authorization : Authorization token (Required)
    X-Machine-Name : The system name (Required)
Response Headers:
    X-Machine-Does-Not-Exist : Machine does not exist
    X-Machine-Not-Available : Machine is not available
    X-Permission-Denied : User does not have permissions to access machine or path
    X-Invalid-Path : targetPath is an invalid path
    X-Timeout : Command has finished with timeout signal
```

Replace `targetPath` with your user's home and run the cell.
<a id='ls-cell'></a>

In [ ]:
targetPath = '/home/llama'
machine = 'cluster'

response = requests.get(
    url=f'{FIRECREST_IP}/utilities/ls',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    params={'targetPath': f'{targetPath}'}
)

handle_response(response)

### Exercise 1: Finding error messages on the response in case of invalid requests

1. Change `targetPath` to an invalid path, run it and find the corresponding error message in the response.
2. Change `targetPath` to another's user home ( `llama` ), where you don't have access, run it and find the corresponding error message in the response.

Run the next cell to see the solutions.

In [ ]:
%cat solutions/ls_errors.txt

### cURL equivalent

```bash
curl -X GET "${FIRECREST_IP}/utilities/ls?targetPath=/home/llama" -H "Authorization: Bearer ${TOKEN}" -H "X-Machine-Name: cluster"
```

## Upload a small file with the blocking call

The second type of calls we are going to look at today is a `POST` request. For this purpose we are going to upload a small file (<5MB) to our cluster. For this exercise you can choose your favorite file or just run the next cell to create one.

In [ ]:
%%writefile files/firecrest_input_file.txt
Hello!

Let's have a look at the request:

### `POST /utilities/upload`

```ini
Form Parameters:
    targetPath (string) : Target path to the location where file will be uploaded to on the {X-Machine-Name} filesystem (Required)
    file : File to be uploaded (Required)
Status Codes:	  
    201 Created : File upload successful
    400 Bad Request : Failed to upload file
Request Headers:
    Authorization : Authorization token (Required)
    X-Machine-Name : The system name (Required)
Response Headers:
    X-Machine-Does-Not-Exist : Machine does not exist
    X-Machine-Not-Available : Machine is not available
    X-Permission-Denied : User does not have permissions to access machine or path
    X-Invalid-Path : targetPath is invalid.
```

We will only set the required arguments: `targetPath`, `X-Machine-Name`, `file` and `Authorization`. Notice that `targetPath` is a **form** parameter and not a **query** parameter as before, so it is passed as a argument in the `data` dictionary.

In order to pass the `file` argument, we don't just give the name of the file as a string; we pass a file object in binary form. For this reason we will open the file with the flags `'rb'`.

In [ ]:
targetPath = '/home/llama'
machine = 'cluster'
localPath = 'files/firecrest_input_file.txt'

response = requests.post(
    url=f'{FIRECREST_IP}/utilities/upload',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    data={'targetPath': targetPath},
    files={'file': open(localPath, 'rb')}
)

handle_response(response)

### cURL equivalent

```bash
curl -X POST "${FIRECREST_IP}/utilities/upload" \
     -H "Authorization: Bearer ${TOKEN}" \
     -H "X-Machine-Name: cluster" \
     -F "targetPath=/home/llama" \
     -F "file=@files/firecrest_input_file.txt"
```

### Exercise:

If you want to make sure you have uploaded your file successfully you can run the [call](#ls-cell) we saw in the previous section.

## Job submission

Before submitting our first job it is important to distinguish between two IDs, slurm’s **job ID** and FirecREST’s **task ID**.

**Slurm’s job ID**
- unique identifier of a slurm job
- it is created by Slurm when the job is submitted
- it can be used to track the state of the job with calls like `squeue` or `sacct`

Every time FirecREST has to interact with slurm it creates a _task_. This _task_ is not necessarily bound to a **job ID**.

**FirecREST’s task ID**
- unique identifier of a FirecREST task
- it is created and updated by FirecREST when the first call regarding this task is created
- it can be used to track the state of the task with the API call we will see later in this section


First let's create locally the job script for our submission. Our job is going to perform a simple SHA-1 calculation of the file we uploaded in the last section.

**Remember to change the output of the script and the location of the `firecrest_input_file.txt` in the cluster so that both paths are in your directory!**

In [ ]:
%%writefile files/firecrest_script.sh
#!/bin/bash

#SBATCH --job-name=test
#SBATCH --output=/home/llama/res.txt
#SBATCH --ntasks=1
#SBATCH --time=10:00

sha1sum /home/llama/firecrest_input_file.txt

For our first job submission we will use two API calls:

1. We will create the FirecREST task of the job submission.
2. We check the status of the task. If the job is submitted correctly we can get the slurm jobid of the job.

The definition of the request is:


### `POST /compute/jobs`

#### Submit Job
```ini
Form Parameters:
    file : Job script of the job (Required)
Status Codes
    201 Created : Task for job creation queued successfully
    400 Bad Request : Failed to submit job file
Request Headers
    Authorization : Authorization token (Required)
    X-Machine-Name : The system name (Required)
Response Headers
    X-Machine-Does-Not-Exist – Machine does not exist
    X-Machine-Not-Available – Machine is not available
    X-Permission-Denied – User does not have permissions to access machine
    X-sbatch-error – sbatch returned error
```

The first call is the following:


In [ ]:
machine = 'cluster'
localPath = 'files/firecrest_script.sh'

response = requests.post(
    url=f'{FIRECREST_IP}/compute/jobs',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    files={'file': open(localPath, 'rb')}
)

handle_response(response)

If everything went well you should get the message `Task created` in the json response. This does **not** necessarily mean that your job is created successfully. This only means that the FirecREST task was created.

Before running the next cell, copy the taskid from the output and set it correctly. In python it should be a string so don't forget the quotes around the task ID.

In [ ]:
# taskid = # Fill this assignment with the correct task_id
taskid = '21b0e954e388c303f07b346722f3fbd8'

response = requests.get(
    url=f'{FIRECREST_IP}/tasks/{taskid}',
    headers={'Authorization': f'Bearer {TOKEN}'}
)

handle_response(response)

If your submission was successful you should get the slurm information in the "data" field.

**All the other fields of the json response are about the FirecREST task and not the scheduler.**

### Exercise:

1. If you want information about all the tasks of your user, not a specific `task_id` then you should make a call to the `/tasks/` endpoint, without any path parameter. Try to fill the request on your own to get information about all you tasks.

<a id='all-tasks-cell'></a>

In [ ]:
response = requests.get(
    headers={'Authorization': f'Bearer {TOKEN}'},
    url= # Fill this assignment with the correct endpoint
)

handle_response(response)

Run the next cell to get the solution.

In [ ]:
%cat solutions/all_tasks.py

## Check for the job's status

As soon as we get the slurm job ID, we can get more information on the progress of that job. The call to the `/compute/jobs/{jobid}` endpoint is going to start a FirecREST task for that purpose.

Just like with the job submission, this is a two-calls process:

1. Make a call to FirecREST to make a new task and get the task's ID.
2. Make a call to FirecREST with this task ID to see its results.

Before running this cell you should set the jobid to the jobid of the job you want to test.

In [ ]:
machine = 'cluster'
# jobid = # Fill this assignment with the correct jobid
jobid = 2

response = requests.get(
    url=f'{FIRECREST_IP}/compute/jobs/{jobid}',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine}
)

handle_response(response)

# response.ok will be True if no error occured
if response.ok:
    taskid = response.json()['task_id']
    
    print(f"\n{50*'.'}")
    time.sleep(1)
    
    response = requests.get(
        url=f'{FIRECREST_IP}/tasks/{taskid}',
        headers={'Authorization': f'Bearer {TOKEN}'}
    )

    handle_response(response)

### Exercises:

1. Try to explain why you (probably) got this error in the second call: `slurm_load_jobs error: Invalid job id specified`
2. Try to remove the sleep from the last cell and see what happens.

Run the next cell to get a hint for question 1.
The second cell includes the solution to both answers.

In [ ]:
%cat solutions/invalid_id_hint.txt

In [ ]:
%cat solutions/job_status.txt

## Job persistent accounting information

If you want persistent information for older jobs you should make a request in the `compute/acct` endpoint like below.

In [ ]:
machine = 'cluster'

response = requests.get(
    url=f'{FIRECREST_IP}/compute/acct?jobs={jobid}',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine}
)

handle_response(response)

# response.ok will be True if no error occured
if response.ok:
    taskid = response.json()['task_id']
    
    print(f"\n{50*'.'}")
    time.sleep(1)
    
    response = requests.get(
        url=f'{FIRECREST_IP}/tasks/{taskid}',
        headers={'Authorization': f'Bearer {TOKEN}'}
    )

    handle_response(response)

### Job output

When FirecREST submits a job on behalf of the user, a directory will be created in the `$HOME` directory of the user in the machine.

The directory is named `firecrest` and its subdirectories are named after the task IDs of the job submissions. In these subdirectories the user can see the job script that was used for the submission as well as the output file(s) if their location is not specified.

## Upload a bigger file through the Storage microservice

For uploading small files the blocking call that we used in a previous section is enough. When the file we want to upload to a machine’s filesystem is bigger than 5MB, we have to use the Storage microservice.

In order for you to do this, you won't upload the file directly through FirecREST but to a staging area. This staging area is the Object Storage and you can get more information [here](https://user.cscs.ch/storage/object_storage/).

This task will be split into more steps but it will correspond to one FirecREST task, so we have to keep track of one task ID.
The steps are:

1. the user asks FirecREST for a link to the staging area
2. the user uploads the file to the staging area
3. FirecREST will poll the staging area and, as soon as the file is uploaded it will transfer it to the filesystem

The first step is to send a `POST` request to FirecREST, to the `/storage/xfer-external/upload` endpoint. Besides the `Authorization` token, we have to include the local path of the file we are going to upload (`sourcePath`) and the target location of the transfer (`targetPath`). Both `sourcePath` and `targetPath` are form data parameters.

> A small tip before running the next cell. You can click on the output of the cell to minimize it and double click to hide it completely.

In [ ]:
targetPath = '/home/llama'
sourcePath = 'files/firecrest_input_file.txt'

response = requests.post(
    url=f'{FIRECREST_IP}/storage/xfer-external/upload',
    headers={'Authorization': f'Bearer {TOKEN}'},
    data={'targetPath': targetPath,
          'sourcePath': sourcePath}
)

handle_response(response)

# response.ok will be True if no error occured
if response.ok:
    taskid = response.json()['task_id']
    
    print(f"\n{50*'.'}")
    time.sleep(1)
    
    response = requests.get(
        url=f'{FIRECREST_IP}/tasks/{taskid}',
        headers={'Authorization': f'Bearer {TOKEN}'}
    )

    handle_response(response)

If the call was successful, the output will be very long. The parts that we care about are the `status` of the FirecREST task, which should be `Form URL from Object Storage received` and the command that FirecREST provides.

In python json objects are just dictionaries, so we can easily isolate the fields we are interested in.

In [ ]:
print(f"Task ID: {response.json()['task']['hash_id']}\n"
      f"Task status code: {response.json()['task']['status']}\n"
      f"Task status description: {response.json()['task']['description']}\n")

# You can isolate the "command" field, that holds the useful information
print(response.json()['task']['data']['msg']['command'])

Copy the command after the exclamation mark in the cell below to execute it.

In [ ]:
# Add the curl command after the exclamation mark and run the cell
! curl ...

### Exercise:

Let's check again on the FirecREST task, we have done it already many times. Make a `GET` request to `/tasks/{taskid}` and get the status of the task. What is the status of the task now?

<a id='get-status-cell'></a>

In [ ]:
# Try to fill the call yourself and run it here.
# If you run the next cell you will get the answer.


In [ ]:
%cat solutions/get_status_id.py


### Exercise: Download the output through the Storage microservice

Just like we did with the `/storage/xfer-external/upload` call, the user will not directly download the file from FirecREST but through a staging area.

The workflow of this task is:

1. The user has to make a request to FirecREST to download the file from `sourcePath`, this should be the absolute path in the `cluster` filesystem. This requires at least two calls from the user:
  1. Make a request to FirecREST to create the task of external uploading.
  2. Poll FirecREST the task until the file has been transfered to the staging area. When it is ready FirecREST will reply with the URL from which we will download it.
2. After receiving the URL you have to download the file with a simple `wget` from the link that will be provided by FirecREST.

#### Step 1A: Make the FirecREST request

In order to fill the request keep in mind that:

- It is a `POST` request.
- The endpoint is `/storage/xfer-external/download`.
- You should pass in the header arguments the `Authorization` token.
- The last argument is `sourcePath` and it a form parameter.


In [ ]:
sourcePath = '/home/llama/res.txt'

response = requests. # Try to fill the request yourself

handle_response(response)

In [ ]:
%cat solutions/external_download.py


#### Step 1B: Based on the reply, check for the status of the task.

You have already done this part in [this](#get-status-cell) exercise. Copy it with the correct `taskid`.

In [ ]:
# Make a request to FirecREST to get the status of the task you just created.


Through python we can isolate the field of the response that we are interested in: 

In [ ]:
print(response.json()['task']['data'])

In [ ]:
# Copy the link and finish the download in bash
!wget -O res.txt "..."


In [ ]:
!cat res.txt

In [ ]:
# This one works for mac
!shasum firecrest_input_file.txt

# This one works for linux
!sha1sum firecrest_input_file.txt

If everything went right you should be getting the same result. Now feel free to create more python cells and experiment with more FirecREST calls.